# Introduction to Quasiperiodic Functions

## An Interactive Undergraduate Tutorial

Welcome to this interactive introduction to quasiperiodic functions! In this notebook, we'll explore:

1. **What are quasiperiodic functions?**
2. **Periodic vs Quasiperiodic motion on the torus**
3. **Connection to quasicrystals**
4. **Triply periodic minimal surfaces**

---

## 1. What are Quasiperiodic Functions?

### Definition

A **quasiperiodic function** is a function that exhibits ordered, almost-periodic behavior but never exactly repeats.

Formally, a function $f: \mathbb{R}^k \to \mathbb{R}$ is **quasiperiodic with $n$ quasiperiods** if it can be written as:

$$f(x_1, \ldots, x_k) = F(\omega_1 \cdot x, \omega_2 \cdot x, \ldots, \omega_n \cdot x)$$

where:
- $F: \mathbb{T}^n \to \mathbb{R}$ is a continuous function on the $n$-torus
- $\omega_1, \ldots, \omega_n \in \mathbb{R}^k$ are **frequency vectors**
- The frequencies are **rationally independent** (no non-trivial rational linear combination equals zero)

### Simple Example: One Variable, Two Frequencies

$$f(x) = \cos(2\pi x) + \cos(2\pi \sqrt{2} x)$$

This function is quasiperiodic because $\sqrt{2}$ is irrational.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
from pathlib import Path

# Add src to path
sys.path.append(str(Path.cwd().parent / 'src'))

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

### Visualize: Periodic vs Quasiperiodic in 1D

In [ ]:
x = np.linspace(0, 20, 2000)

# Periodic function (commensurate frequencies)
f_periodic = np.cos(2*np.pi*x) + np.cos(2*np.pi*2*x)

# Quasiperiodic function (incommensurate frequencies)
f_quasiperiodic = np.cos(2*np.pi*x) + np.cos(2*np.pi*np.sqrt(2)*x)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))

ax1.plot(x, f_periodic, 'b-', linewidth=1.5)
ax1.set_title('Periodic: f(x) = cos(2πx) + cos(4πx)', fontsize=14, fontweight='bold')
ax1.set_ylabel('f(x)', fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.axhline(y=0, color='k', linewidth=0.5)

ax2.plot(x, f_quasiperiodic, 'r-', linewidth=1.5)
ax2.set_title('Quasiperiodic: f(x) = cos(2πx) + cos(2π√2 x)', fontsize=14, fontweight='bold')
ax2.set_xlabel('x', fontsize=12)
ax2.set_ylabel('f(x)', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0, color='k', linewidth=0.5)

plt.tight_layout()
plt.show()

print("Notice: The periodic function repeats exactly, while the quasiperiodic function")
print("shows ordered structure but never exactly repeats!")

## 2. Motion on the 2-Torus

### The 2-Torus as a Phase Space

The 2-torus $\mathbb{T}^2$ can be parametrized by two angles $(\theta, \phi)$ where each ranges from $0$ to $2\pi$.

In 3D, the torus is given by:
$$\begin{align}
x &= (R + r\cos\theta)\cos\phi \\
y &= (R + r\cos\theta)\sin\phi \\
z &= r\sin\theta
\end{align}$$

where $R$ is the major radius and $r$ is the minor radius.

### Trajectories with Winding Number $\alpha$

Consider a trajectory that evolves as:
$$\begin{align}
\theta(t) &= t \\
\phi(t) &= \alpha t
\end{align}$$

- If $\alpha = p/q$ is **rational**, the trajectory closes after $q$ loops (periodic)
- If $\alpha$ is **irrational**, the trajectory densely fills the torus (quasiperiodic)

In [ ]:
from utils.math_functions import torus_surface, torus_trajectory, golden_ratio
from utils.plotting_helpers import create_torus_surface_plotly, create_trajectory_plotly

# Create torus surface
X_torus, Y_torus, Z_torus = torus_surface(R=2, r=1)

# Rational winding number: α = 2/3
alpha_rational = 2/3
t = np.linspace(0, 100, 10000)
X_rat, Y_rat, Z_rat = torus_trajectory(t, alpha_rational)

# Irrational winding number: α = φ - 1 (golden ratio minus 1)
alpha_irrational = golden_ratio() - 1
X_irr, Y_irr, Z_irr = torus_trajectory(t, alpha_irrational)

# Create comparison plot
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        f'Rational: α = 2/3 (Periodic)',
        f'Irrational: α ≈ {alpha_irrational:.6f} (Quasiperiodic)'
    ),
    specs=[[{'type': 'surface'}, {'type': 'surface'}]],
)

# Left: Rational (periodic)
fig.add_trace(
    go.Surface(x=X_torus, y=Y_torus, z=Z_torus, opacity=0.2, 
               colorscale='Blues', showscale=False),
    row=1, col=1
)
fig.add_trace(
    go.Scatter3d(x=X_rat, y=Y_rat, z=Z_rat, mode='lines',
                 line=dict(color='red', width=5), name='Closed Orbit'),
    row=1, col=1
)

# Right: Irrational (quasiperiodic)
fig.add_trace(
    go.Surface(x=X_torus, y=Y_torus, z=Z_torus, opacity=0.2,
               colorscale='Blues', showscale=False),
    row=1, col=2
)
fig.add_trace(
    go.Scatter3d(x=X_irr, y=Y_irr, z=Z_irr, mode='lines',
                 line=dict(color='green', width=3), name='Dense Trajectory'),
    row=1, col=2
)

fig.update_layout(
    height=600,
    width=1200,
    title_text="<b>Periodic vs Quasiperiodic Motion on the Torus</b>"
)
fig.update_scenes(aspectmode='data')

fig.show()

### Poincaré Section

A **Poincaré section** is a powerful tool to visualize the difference:

- **Rational α**: Finitely many points (one for each loop)
- **Irrational α**: Dense circle (fills continuously)

In [ ]:
from utils.math_functions import poincare_section

# Compute Poincaré sections
t_long = np.linspace(0, 1000, 100000)

theta_rat, phi_rat = poincare_section(t_long, alpha_rational)
theta_irr, phi_irr = poincare_section(t_long, alpha_irrational)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

ax1.scatter(theta_rat, phi_rat, c='red', s=50, alpha=0.8)
ax1.set_title(f'Poincaré Section: α = 2/3 (Rational)', fontsize=14, fontweight='bold')
ax1.set_xlabel('θ (poloidal)', fontsize=12)
ax1.set_ylabel('φ (toroidal)', fontsize=12)
ax1.set_xlim(0, 2*np.pi)
ax1.set_ylim(0, 2*np.pi)
ax1.grid(True, alpha=0.3)
ax1.text(0.5, 0.95, 'Exactly 3 points!', transform=ax1.transAxes,
         fontsize=12, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
         horizontalalignment='center', verticalalignment='top')

ax2.scatter(theta_irr, phi_irr, c='green', s=2, alpha=0.5)
ax2.set_title(f'Poincaré Section: α ≈ {alpha_irrational:.4f} (Irrational)', 
              fontsize=14, fontweight='bold')
ax2.set_xlabel('θ (poloidal)', fontsize=12)
ax2.set_ylabel('φ (toroidal)', fontsize=12)
ax2.set_xlim(0, 2*np.pi)
ax2.set_ylim(0, 2*np.pi)
ax2.grid(True, alpha=0.3)
ax2.text(0.5, 0.95, 'Dense circle!', transform=ax2.transAxes,
         fontsize=12, bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5),
         horizontalalignment='center', verticalalignment='top')

plt.tight_layout()
plt.show()

## 3. Connection to Quasicrystals

### Penrose Tilings

Quasiperiodic functions are intimately connected to **quasicrystals** - materials discovered by Dan Shechtman in 1982 (Nobel Prize 2011).

**Penrose tilings** are 2D quasiperiodic patterns with 5-fold rotational symmetry (impossible for periodic crystals!)

Key properties:
- **Aperiodic**: Never repeats exactly
- **Ordered**: Has long-range order
- **Self-similar**: Contains patterns at all scales

In [ ]:
from visualizations.quasicrystals import visualize_penrose_tiling

# Generate Penrose tiling
fig = visualize_penrose_tiling(iterations=6, scale=100)
fig.show()

print("\nNotice the 5-fold rotational symmetry - impossible for periodic crystals!")

### Cut-and-Project Method

One way to generate quasicrystals is the **cut-and-project method**:

1. Start with a periodic lattice in higher dimensions
2. Define a "window" in a non-rational direction
3. Project points within the window to lower dimensions

This creates a quasiperiodic point set!

In [ ]:
from visualizations.quasicrystals import visualize_cut_and_project

# Visualize cut-and-project with golden ratio slope
fig = visualize_cut_and_project(slope=golden_ratio(), n_points=2000, width=0.3)
fig.show()

## 4. Triply Periodic Minimal Surfaces

### The Schoen I-WP Surface

The function:
$$F(x,y,z) = \cos(x)\cos(y) + \cos(y)\cos(z) + \cos(z)\cos(x)$$

approximates the **Schoen I-WP** ("wrapped package") minimal surface.

Properties:
- **Triply periodic**: Repeats in all three directions with period $2\pi$
- **Genus 4**: Topologically equivalent to a sphere with 4 handles
- **Minimal surface**: Zero mean curvature (locally minimizes area)

When we take planar sections of this 3D periodic surface along **irrational directions**, we get **quasiperiodic patterns**!

In [ ]:
from visualizations.minimal_surfaces import visualize_minimal_surface

# Visualize Schoen I-WP surface at level 0
fig = visualize_minimal_surface('schoen-iwp', level=0, resolution=60)
fig.show()

print("\nThis surface approximates a minimal surface with genus 4!")

### Compare Different Minimal Surfaces

In [ ]:
from visualizations.minimal_surfaces import compare_minimal_surfaces

fig = compare_minimal_surfaces(level=0, resolution=50)
fig.show()

## Summary

We've explored:

1. ✅ **Quasiperiodic functions**: Functions that are ordered but never repeat
2. ✅ **Torus dynamics**: Rational winding = periodic, irrational = quasiperiodic
3. ✅ **Quasicrystals**: Physical manifestations of quasiperiodicity (Penrose tilings)
4. ✅ **Minimal surfaces**: 3D periodic structures whose sections give quasiperiodic patterns

### Key Takeaway

**Quasiperiodic systems bridge the gap between order and chaos:**
- More ordered than random
- More complex than periodic
- Appear in nature (quasicrystals, music, solar system dynamics)

---

## Exercises

1. **Experiment**: Change the winding number α in the torus visualization. What values give closed orbits?

2. **Mathematical**: Prove that if α = p/q (rational), the orbit closes after exactly q loops.

3. **Computational**: Generate a 2D quasiperiodic function using 3 incommensurate frequencies.

4. **Exploration**: What happens to the Schoen I-WP surface as you change the level parameter?

---

## Next Steps

Continue to:
- `02_advanced_torus_dynamics.ipynb` - Deeper dive into torus dynamics
- `03_minimal_surface_topology.ipynb` - Topology of level sets
- `04_quasicrystal_physics.ipynb` - Physical properties of quasicrystals

Or run the interactive Dash app:
```bash
python app/dash_torus_app.py
```